In [7]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [8]:
url = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1"
webpage = requests.get("https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1").text

In [9]:
soup = BeautifulSoup(webpage,'lxml')

In [10]:
movie_containers = soup.find_all("div", class_="lister-item mode-advanced")
movies = []
for container in movie_containers:
    sno = container.find("span", class_="lister-item-index").text.strip(".")
    movie_header = container.find("h3", class_="lister-item-header")
    movie_name = movie_header.a.text

    directors = container.find("p", class_="").find_all("a")
    director_names = [director.text for director in directors]

    duration = container.find("span", class_="runtime").text.strip(" min")

    year = movie_header.find("span", class_="lister-item-year").text.strip("()")

    ratings = container.find("div", class_="inline-block ratings-imdb-rating").strong.text

    metascore = container.find("span", class_="metascore favorable")
    metascore = metascore.text.strip() if metascore else "N/A"

    movie_details = {
        "Sno": sno,
        "Movie Name": movie_name,
        "Duration": duration,
        "Year": year,
        "Ratings": ratings,
        "Metascore": metascore
    }
    for i, director_name in enumerate(director_names):
        movie_details[f"Director{i+1}"] = director_name

    movies.append(movie_details)

In [11]:
first_csv = pd.DataFrame(movies)
first_csv

,Sno,Movie Name,Duration,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6,Director7
0,1,Spider-Man: Across the Spider-Verse,140,2023,9.0,86,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez
1,2,The Dark Knight,152,2008,9.0,84,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,NaN,NaN
2,3,The Lord of the Rings: The Return of the King,201,2003,9.0,94,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,NaN,NaN
3,4,Inception,148,2010,8.8,74,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,NaN,NaN
4,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,NaN,NaN
5,6,The Lord of the Rings: The Two Towers,179,2002,8.8,87,Peter Jackson,Elijah Wood,Ian McKellen,Viggo Mortensen,Orlando Bloom,NaN,NaN
6,7,The Matrix,136,1999,8.7,73,Lana Wachowski,Lilly Wachowski,Keanu Reeves,Laurence Fishburne,Carrie-Anne Moss,Hugo Weaving,NaN
7,8,Star Wars: Episode V - The Empire Strikes Back,124,1980,8.7,82,Irvin Kershner,Mark Hamill,Harrison Ford,Carrie Fisher,Billy Dee Williams,NaN,NaN
8,9,Soorarai Pottru,153,2020,8.7,N/A,Sudha Kongara,Suriya,Paresh Rawal,Aparna Balamurali,Urvashi,NaN,NaN
9,10,Terminator 2: Judgment Day,137,1991,8.6,75,James Cameron,Arnold Schwarzenegger,Linda Hamilton,Edward Furlong,Robert Patrick,NaN,NaN


In [12]:
movies2 = []
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

movie_containers = soup.find_all("div", class_="lister-item mode-advanced")
for container in movie_containers:
    movie_header = container.find("h3", class_="lister-item-header")
    movie_name = movie_header.a.text

    stars_container = container.find("p", class_="").find_all("a")
    stars = [star.text for star in stars_container]

    votes = container.find("p", class_="sort-num_votes-visible").find("span", attrs={"name": "nv"}).text

    genres_container = container.find("span", class_="genre").text.strip().split(", ")
    genres = [genre.strip() for genre in genres_container]

    gross_container = container.find("p", class_="sort-num_votes-visible").find_next_sibling("span")
    gross_collection = gross_container.text.strip() if gross_container else "N/A"

    popularity = container.find("span", class_="lister-item-index unbold text-primary").text.strip(".")

    certification = container.find("span", class_="certificate").text.strip() if container.find("span", class_="certificate") else "N/A"

    movie_details2 = {
        "Movie Name": movie_name,
        "Votes": votes,
        "Gross Collection": gross_collection,
        "Popularity": popularity,
        "Certification": certification
    }
    for i, star_name in enumerate(stars):
        movie_details2[f"Star{i+1}"] = star_name
    for i, genre_name in enumerate(genres):
        movie_details2[f"Genre{i+1}"] = genre_name

    movies2.append(movie_details2)


In [13]:
second_csv = pd.DataFrame(movies2)
second_csv

,Movie Name,Votes,Gross Collection,Popularity,Certification,Star1,Star2,Star3,Star4,Star5,Star6,Star7,Genre1,Genre2,Genre3
0,Spider-Man: Across the Spider-Verse,"132,708",N/A,1,PG,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,Animation,Action,Adventure
1,The Dark Knight,"2,726,578",N/A,2,PG-13,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,NaN,NaN,Action,Crime,Drama
2,The Lord of the Rings: The Return of the King,"1,891,618",N/A,3,PG-13,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,NaN,NaN,Action,Adventure,Drama
3,Inception,"2,419,863",N/A,4,PG-13,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,NaN,NaN,Action,Adventure,Sci-Fi
4,The Lord of the Rings: The Fellowship of the Ring,"1,920,174",N/A,5,PG-13,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,NaN,NaN,Action,Adventure,Drama
5,The Lord of the Rings: The Two Towers,"1,707,202",N/A,6,PG-13,Peter Jackson,Elijah Wood,Ian McKellen,Viggo Mortensen,Orlando Bloom,NaN,NaN,Action,Adventure,Drama
6,The Matrix,"1,962,895",N/A,7,R,Lana Wachowski,Lilly Wachowski,Keanu Reeves,Laurence Fishburne,Carrie-Anne Moss,Hugo Weaving,NaN,Action,Sci-Fi,NaN
7,Star Wars: Episode V - The Empire Strikes Back,"1,324,407",N/A,8,PG,Irvin Kershner,Mark Hamill,Harrison Ford,Carrie Fisher,Billy Dee Williams,NaN,NaN,Action,Adventure,Fantasy
8,Soorarai Pottru,"119,676",N/A,9,TV-MA,Sudha Kongara,Suriya,Paresh Rawal,Aparna Balamurali,Urvashi,NaN,NaN,Action,Drama,NaN
9,Terminator 2: Judgment Day,"1,125,961",N/A,10,R,James Cameron,Arnold Schwarzenegger,Linda Hamilton,Edward Furlong,Robert Patrick,NaN,NaN,Action,Sci-Fi,NaN


# Phase 2

In [14]:
import sqlite3
import csv

# Connect to the SQLite database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Create the Movies table
cursor.execute('''
    CREATE TABLE movies_ (
        Sno INTEGER,
        MovieName TEXT,
        DirectorName TEXT,
        Duration INTEGER,
        Genre TEXT,
        Ratings REAL
    )
''')

# Import data from CSV file to Movies table
with open('final_df.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        cursor.execute('INSERT INTO Movies VALUES (?, ?, ?, ?, ?, ?)', row)

# Commit the changes
conn.commit()

# Create the MovieDetails table
cursor.execute('''
    CREATE TABLE MovieDetails (
        MovieName TEXT,
        Stars TEXT,
        Votes INTEGER,
        Genre TEXT,
        GrossCollection REAL,
        Popularity REAL,
        Certification TEXT
    )
''')

# Import data from CSV file to MovieDetails table
with open('final_df.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        cursor.execute('INSERT INTO MovieDetails VALUES (?, ?, ?, ?, ?, ?, ?)', row)

# Commit the changes
conn.commit()

# Close the connection
conn.close()


OperationalError: ignored

# Query For Table 1

In [14]:
SELECT *
FROM Movies
WHERE DirectorName IN ('Christopher', 'Matt Reeves');


In [ ]:
SELECT *
FROM Movies
WHERE Duration BETWEEN 140 AND 190;


In [ ]:
SELECT *
FROM Movies
WHERE Ratings > 7
ORDER BY Ratings ASC;


In [ ]:
SELECT MovieName
FROM Movies
ORDER BY MovieName DESC;


In [ ]:
SELECT MovieName
FROM Movies
WHERE MovieName LIKE 'P%' AND Ratings > 7;


# Query for Table 2

In [ ]:
SELECT MovieName
FROM MovieDetails
WHERE Stars LIKE '%Arnold Schwarzenegger%'
ORDER BY MovieName ASC;


In [ ]:
SELECT *
FROM MovieDetails
WHERE Votes = (SELECT MAX(Votes) FROM MovieDetails);


In [ ]:
SELECT MovieName
FROM MovieDetails
ORDER BY GrossCollection DESC;


In [ ]:
SELECT MovieName, GrossCollection
FROM MovieDetails
WHERE Stars LIKE '%Arnold%'


In [ ]:
SELECT *
FROM MovieDetails
WHERE Genre LIKE '%comedy%' AND Genre LIKE '%action%';
